In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [3]:
train_dir = "C:/Users/lenovo/Desktop/devquest/train"
val_dir = "C:/Users/lenovo/Desktop/devquest/validation"

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
val_datagen = ImageDataGenerator(rescale=1./255)

# Loading the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=8,  # Reduce batch size
    class_mode='categorical'
)

# Loading the validation data
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3744 images belonging to 6 classes.
Found 91 images belonging to 6 classes.


In [9]:
# Load the pre-trained mobilenetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:50]:  # Freeze only the first 50 layers
    layer.trainable = False


from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()
from tensorflow.keras.callbacks import EarlyStopping  # Ensure the import is present

# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitors validation loss
    patience=3,          # Stops training after 3 epochs with no improvement
    restore_best_weights=True  # Restores model weights from the best epoch
)

class_weight = {0: 1.0, 1: 2.0, 2: 2.0, 3: 1.0, 4: 1.5, 5: 1.5}  # Adjust based on class distribution
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    class_weight=class_weight,
    callbacks=[early_stopping]
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           1,542 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,588,486 (9.87 MB)

 Trainable params: 2,504,582 (9.55 MB)

 Non-trainable params: 83,904 (327.75 KB)

C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 375s 612ms/step - accuracy: 0.2487 - loss: 3.6021 - val_accuracy: 0.3516 - val_loss: 2.5000
Epoch 2/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 307s 580ms/step - accuracy: 0.4127 - loss: 2.4408 - val_accuracy: 0.5165 - val_loss: 1.6993
Epoch 3/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 385s 823ms/step - accuracy: 0.4595 - loss: 2.1283 - val_accuracy: 0.6374 - val_loss: 1.1305
Epoch 4/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 461s 984ms/step - accuracy: 0.5320 - loss: 1.8666 - val_accuracy: 0.6703 - val_loss: 0.8921
Epoch 5/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 460s 983ms/step - accuracy: 0.5577 - loss: 1.7356 - val_accuracy: 0.7692 - val_loss: 0.7367
Epoch 6/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 483s 1s/step - accuracy: 0.6016 - loss: 1.5880 - val_accuracy: 0.7802 - val_loss: 0.6942
Epoch 7/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 490s 1s/step - accuracy: 0.6340 - loss: 1.4739 - val_accuracy: 0.7912 - val_loss: 0.6298
Epoch 8/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 488s 1s/step - accuracy: 0.6514 - loss: 1.

In [17]:
# Save the trained model
model.save('my_trainedmodel.h5')


In [19]:
# Save your model in the native Keras format
model.save('my_trained_model.keras')
  # Save as an HDF5 file


In [21]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 524ms/step - accuracy: 0.9241 - loss: 0.3488
Validation Loss: 0.3304397761821747, Validation Accuracy: 0.9340659379959106


In [68]:
def predict_nail_condition(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    
    # Predict the class
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions, axis=1)

    # Map class_idx to the class name
    class_names = ['acrall lentiginous melanoma', 'blue finger', 'onychogryphosis', 'healthy nail', 'clubbing', 'pitting']
    predicted_class = class_names[class_idx[0]]
    print(f'The predicted nail condition is: {predicted_class}')
    return predicted_class

In [70]:
image_path = 'test9.png'  # Replace with the path to your image
predict_nail_condition(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The predicted nail condition is: acrall lentiginous melanoma


'acrall lentiginous melanoma'